In [15]:
import torch
import torch.nn as nn
import string
import random
import sys
import unidecode
from datetime import datetime
# from torch.utils.tensorboard import SummaryWriter


In [16]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
# get characters
all_characters = string.printable
n_characters = len(all_characters)

In [18]:
# read file
file = unidecode.unidecode(open('./data/alice_in_wonderland.txt').read())



In [19]:
# module

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        print('--'*30)
        print(x.shape)
        print(hidden.shape)
        print(cell.shape)
        print('--'*30)
        out = self.embed(x)
        out, (hidden, cell) = self.lstm(out.unsqueeze(1), (hidden, cell))
        out = self.fc(out.reshape(out.shape[0], -1))
        return out, (hidden, cell)
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, cell



In [20]:
class Generator():
    def __init__(self):
        self.chunk_len = 250
        self.num_epochs = 3000
        self.batch_size = 1
        self.print_every = self.num_epochs // 30 or 1
        self.hidden_size = 256
        self.num_layers = 2
        self.lr = 0.003


    def char_tensor(self, string):
        tensor = torch.zeros(len(string)).long()
        for c in range(len(string)):
            tensor[c] = all_characters.index(string[c])
        return tensor


    def get_random_batch(self):
        start_idx = random.randint(0, len(file) - self.chunk_len)
        end_idx = start_idx + self.chunk_len + 1
        text_str = file[start_idx:end_idx]
        text_input = torch.zeros(self.batch_size, self.chunk_len)
        text_target = torch.zeros(self.batch_size, self.chunk_len)
        for i in range(self.batch_size):
            text_input[i,:] = self.char_tensor(text_str[:-1])
            text_target[i,:] = self.char_tensor(text_str[1:])
        return text_input.long(), text_target.long()


    def generate(self, initial_str='i would like', predict_len=200, temperature=0.85):
        hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)
        initial_input = self.char_tensor(initial_str)
        predicted = initial_str
        
        for p in range(len(initial_str) - 1):
            _, (hidden, cell) = self.rnn(initial_input[p].view(1).to(device), hidden, cell)

        last_char = initial_input[-1]
        for p in range(predict_len):
            output, (hidden, cell) = self.rnn(last_char.view(1).to(device), hidden, cell)
            output_dist = output.data.view(-1).div(temperature).exp()
            top_char = torch.multinomial(output_dist, 1)[0]
            predicted_char = all_characters[top_char]
            predicted += predicted_char
            last_char = self.char_tensor(predicted_char)

        return predicted


    def train(self):
        self.rnn = RNN(n_characters, self.hidden_size, self.num_layers, n_characters).to(device)
        optimizer = torch.optim.Adam(self.rnn.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()
        # writer = SummaryWriter(f'runs/alice0')
        print(f'<{datetime.now()}>starting training')

        for epoch in range(1, self.num_epochs + 1):
            input, target = self.get_random_batch()
            hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)

            self.rnn.zero_grad()
            loss = 0
            input = input.to(device)
            target = target.to(device)

            for c in range(self.chunk_len):
                output, (hidden, cell) = self.rnn(input[:, c], hidden, cell)
                loss += criterion(output, target[:, c])

            loss.backward()
            optimizer.step()
            loss = loss.item() / self.chunk_len

            if epoch % self.print_every == 0:
                print(f'\n\n<{datetime.now()}> | epoch: {epoch}/{self.num_epochs} | loss: {loss}')
                print(self.generate())


In [21]:
gentext = Generator()
gentext.train()


<2023-05-29 10:12:27.438213>starting training
------------------------------------------------------------
torch.Size([1])
torch.Size([2, 1, 256])
torch.Size([2, 1, 256])
------------------------------------------------------------
------------------------------------------------------------
torch.Size([1])
torch.Size([2, 1, 256])
torch.Size([2, 1, 256])
------------------------------------------------------------
------------------------------------------------------------
torch.Size([1])
torch.Size([2, 1, 256])
torch.Size([2, 1, 256])
------------------------------------------------------------
------------------------------------------------------------
torch.Size([1])
torch.Size([2, 1, 256])
torch.Size([2, 1, 256])
------------------------------------------------------------
------------------------------------------------------------
torch.Size([1])
torch.Size([2, 1, 256])
torch.Size([2, 1, 256])
------------------------------------------------------------
------------------------

KeyboardInterrupt: 

In [ ]:
gentext.generate(initial_str='i would like', predict_len=400, temperature=0.65)


'i would like with so minutes of the Mouse, you know,\n                      Pear, on the might time a tone, so she slong of and generation or thing was how.  This she had been and moment again!  Then when she sad of the midest the right should not bake as she put on it make in a little and foot.  `We wanted as she can impining of little in the robster had nearly at the would not had got of the top of a very ar'

In [ ]:
# print(f'<{datetime.now()}> Training started, doing {epochs} epochs on {corpus_size} articles')
